# Introduction to OSPF
<i>Adapted for use with FABRIC from [OSPF](https://www.cs.unc.edu/Research/geni/geniEdu/06-Ospf.html)</i>
    
In this tutorial you will experiment with the OSPF routing protocol.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)
    
OSPF stands for Open Shortest Path First, and is currently the most widely used interior gateway protocol in the world. OSPF is a dynamic routing protocol, meaning changes are accounted for automatically. Hello messages are used to discover neighbors and find breaks. By default, hello messages are sent by OSPF routers every 10 seconds on point to point segments.
<br><br>Each OSPF router maintains a Link State Database (LSDB) containing Link State Advertisements (LSAs). Newly discovered neighbors exchange their LSDBs with Database Description (DDs) messages.
<br><br>OSPF uses Router Dead Intervals, the maximum time a router may be "silent" before being declared "down". OSPF uses Djikstra's Shortest path algorithm to compute routes.

## 1. Design the Experiment
### 1.1 Reserve Resources
A topology with 4 nodes to be setup as OSPF routers with 4 connections setup in a ring.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name="OSPF")
    
    #Node A
    A = slice.add_node(name="A", site="MAX")
    A.set_image('default_ubuntu_20')
    AP1 = A.add_component(model='NIC_Basic', name="APort1").get_interfaces()[0] 
    AP2 = A.add_component(model='NIC_Basic', name="APort2").get_interfaces()[0]
    
    #Node B
    B = slice.add_node(name="B", site="MAX")
    B.set_image('default_ubuntu_20')
    BP1 = B.add_component(model='NIC_Basic', name="BPort1").get_interfaces()[0] 
    BP2 = B.add_component(model='NIC_Basic', name="BPort2").get_interfaces()[0]
    
    #Node C
    C = slice.add_node(name="C", site="MAX")
    C.set_image('default_ubuntu_20')
    CP1 = C.add_component(model='NIC_Basic', name="CPort1").get_interfaces()[0] 
    CP2 = C.add_component(model='NIC_Basic', name="CPort2").get_interfaces()[0]
    
    #Node D
    D = slice.add_node(name="D", site="MAX")
    D.set_image('default_ubuntu_20')
    DP1 = D.add_component(model='NIC_Basic', name="DPort1").get_interfaces()[0] 
    DP2 = D.add_component(model='NIC_Basic', name="DPort2").get_interfaces()[0]
    
    Lan1 = slice.add_l2network(name="Lan1", interfaces=[AP1, BP1])
    Lan2 = slice.add_l2network(name="Lan2", interfaces=[BP2, CP1])
    Lan3 = slice.add_l2network(name="Lan3", interfaces=[CP2, DP1])
    Lan4 = slice.add_l2network(name="Lan4", interfaces=[DP2, AP2])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set Routes
Setup connections between nodes on 4 seperate networks.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    A = slice.get_node(name="A") 
    B = slice.get_node(name="B")
    C = slice.get_node(name="C")
    D = slice.get_node(name="D")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("10.1.2.0/24")
    subnet3 = IPv4Network("10.1.3.0/24")
    subnet4 = IPv4Network("10.1.4.0/24")
    
    AP1 = A.get_interface(network_name="Lan1") 
    AP1.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
    
    BP1 = B.get_interface(network_name="Lan1") 
    BP1.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    
    BP2 = B.get_interface(network_name="Lan2") 
    BP2.ip_addr_add(addr="10.1.2.1", subnet=subnet2)
            
    CP1 = C.get_interface(network_name="Lan2") 
    CP1.ip_addr_add(addr="10.1.2.2", subnet=subnet2)
    
    CP2 = C.get_interface(network_name="Lan3") 
    CP2.ip_addr_add(addr="10.1.3.1", subnet=subnet3)
    
    DP1 = D.get_interface(network_name="Lan3") 
    DP1.ip_addr_add(addr="10.1.3.2", subnet=subnet3)
    
    DP2 = D.get_interface(network_name="Lan4") 
    DP2.ip_addr_add(addr="10.1.4.1", subnet=subnet4)
            
    AP2 = A.get_interface(network_name="Lan4") 
    AP2.ip_addr_add(addr="10.1.4.2", subnet=subnet4)
except Exception as e:
    print(f"Exception: {e}")

### 1.3 Installation Script
Upload and execute installation script on all nodes. The OSPF and Zebra services are then started.
Read through the installation script [Here](./scripts/ospf-script.sh).

In [ ]:
for node in slice.get_nodes():
    node.upload_file("scripts/ospf-script.sh","ospf-script.sh")
    node.execute("chmod +x ospf-script.sh;\
    ./ospf-script.sh;\
    sudo cp /usr/share/doc/quagga-core/examples/vtysh.conf.sample /etc/quagga/vtysh.conf;\
    sudo cp /usr/share/doc/quagga-core/examples/bgpd.conf.sample /etc/quagga/bgpd.conf;\
    sudo chown quagga:quagga /etc/quagga/*.conf;\
    sudo chown quagga:quaggavty /etc/quagga/vtysh.conf;\
    sudo chmod 640 /etc/quagga/*.conf;\
    sudo service zebra start;\
    sudo service ospfd start")


## 2. Experiment
### 2.1 Router Interface
1. Login to each node
2. Open the Quagga shell
Quagga provides an integreated user inteface shell called vtysh, a cisco-like CLI, connecting to the underlying daemons (zebra and ospfd). <br>Open the shell on each node by typing:
<br>```sudo vtysh```
3. Look at the OSPF neighbors for a node by typing the following into a vtysh console:
<br>```show ip ospf neighbor```
<br>The output shows the status of neighbors including details of the connections.

4. The routing table for a node can be seen by typing the following into a vtysh console:
<br>```show ip ospf route```
<br>This table shows how packets would be forwarded from the source to a given destination.
5. Exiting the console can be done by executing:
<br>```exit```


### 2.2 Dead Link
1. Login to a node
2. Show available interfaces with:
<br>```ip addr```
3. Take down one of the links (ens7 or ens8), keep in mind the IP address associated:
<br>```sudo ip link set dev <interface> down```

4. Quickly switch back to vtysh shell and reissue:
<br>```show ip ospf neighbor```
5. Reissue the command multiple times and watch the Dead Time count down until the neighbor is removed from the table. The other link will also count down, but resets when receiving a hello message.
6. Exit the console and bring the interface back up by typing:
<br>```sudo ip link set dev <interface> up```
7. List the OSPF neighbors once more to see the link listed.

### 2.3 Route Change
1. Login to node 'A'
2. Run the following command:
<br>```traceroute 10.1.3.1```
<br>Traceroute will determine the route that packets are taking through the network. Running the command on node A will show packets travelling through 10.1.4.1 (node D) to 10.1.3.1 (node C).
3. Take down the interface to node D (ens8):
<br>```sudo ip link set dev <interface> down```

4. Run traceroute again:
<br>```traceroute 10.1.3.1```
<br>The output should now show traffic going through 10.1.1.2 (node B).
5. Bring the route to node D (ens8) back up:
<br>```sudo ip link set dev <interface> up```
6. Repeatedly run traceroute and eventually the route through node D will reappear.

## 3. Cleanup Resources
### 3.1 Delete Slice

In [ ]:
try:
    slice = fablib.get_slice("OSPF")
    slice.delete()
    
except Exception as e:
    print(f"Fail: {e}")